# Galactic Chemical Evolution Activity

In this notebook, you will use OMEGA+
([Côté et al. 2018](http://adsabs.harvard.edu/abs/2018ApJ...859...67C)), a simple galactic chemical evolution code designed to connect nuclear astrophysics with astronomical observations.

**Open-source Python packages**
* [NuPyCEE](https://github.com/NuGrid/NuPyCEE) - NuGrid Python Chemical Evolution Environment
* [JINAPyCEE](https://github.com/becot85/JINAPyCEE) - JINA Python Chemical Evolution Environment

In [ ]:
# Import the OMEGA+ code and standard packages
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from JINAPyCEE import omega_plus
from NuPyCEE import stellab

# 1. Star Formation and Gas Reservoirs

Here you will learn about the interplay between galactic inflows, star formation, and galactic outflows. The model consists of a galaxy surrounded by a large gas reservoir called the circumgalactic medium. The galaxy is composed of gas and stars. The model follows galactic inflows, representing the transfer of gas from the circumgalactic medium into the galaxy, as well as galactic outflows, representing the transfer of gas from the galaxy into the circumgalactic medium.

<img src="inflow_outflow.tiff" alt="drawing" width="350"/>

The simulation will start with $1.56\times10^{11}$ M$_\odot$ of gas inside the circumgalactic medium. Once gas will be introduced inside the galaxy, stars will start to form.  The equation describing the rate of change of the mass of gas **inside** the galaxy is

$$\dot{M}_\mathrm{gas,in} = \dot{M}_\mathrm{inflow} - \dot{M}_\star + \dot{M}_\mathrm{ej} - \dot{M}_\mathrm{outflow} \mathrm{\quad[M_\odot/yr]}$$

* $M_\mathrm{gas,in}$: Mass of gas inside the galaxy.
* $\dot{M}_\mathrm{inflow}$: Galactic inflow rate.
* $\dot{M}_\star$: Star formation rate. Conversion of gas into stars.
* $\dot{M}_\mathrm{ej}$: Stellar ejecta. Mass ejected by stars when they die (ex. supernova explosion).
* $\dot{M}_\mathrm{outflow}$: Galactic outflow rate.

### Run the Model
Run the cell below to launch the model. **For now, simply run the model without changing the parameters**.

In [ ]:
# Star formation efficiency.
# This is controling how fast the gas inside the galaxy is converted into stars.
# You can vary this parameter between 0.001 and 0.05
sfe = 0.0

# Galactic outflow rate.
# You can vary this parameter between 0.0 and 5.0
mass_loading = 0.0

# !!Do not modify the following line!!
# Run OMEGA+ with the first set of parameters
o = omega_plus.omega_plus(sfe=sfe, mass_loading=mass_loading, special_timesteps=60, \
                          table='yield_tables/agb_and_massive_stars_K10_K06_0.5HNe.txt', \
                          t_inflow=5e9, t_star=1.0e8, f_halo_to_gal_out=0.0)

### Extract the Information

In [ ]:
# The equations describing the evolution of the galaxy are solved step by step.

# Age of the galaxy at each timestep [Gyr, billion of years].
t_sim = o.inner.history.age[:-1]

# Print the number of timesteps.
print(o.inner.nb_timesteps)

In [ ]:
# The mass of the two gas reservoirs is decomposed into isotopes.
# To calculate the total mass, at each timestep, you must sum the mass of all isotopes

# Print the number of isotopes.
print('Number of isotopes =',o.inner.nb_isotopes)

# Print the list of isotopes
print('List of isotopes =',o.inner.history.isotopes)

**Here are the important arrays**:
* Mass of isotopes in the gas inside the circumgalactic medium, for each timestep
    * **o.ymgal_outer**[ timestep index ][ isotope index ]
* Mass of isotopes in the gas inside the galaxy, for each timestep
    * **o.inner.ymgal**[ timestep index ][ isotope index ]
* Mass of gas converted into stars at each timestep (**this is not the total cumulated mass of stars**)
    * **o.inner.history.m_locked**[ timestep index ]
* Mass of isotopes ejected by stars at each timestep
    * **o.inner.mdot**[ timestep index ][ isotope index ]

In [ ]:
# As an example, here is the mass of He-4 in the circumgalactic medium at the 3rd timestep
i_step = 2
i_isotope = o.inner.history.isotopes.index('He-4')
print(o.ymgal_outer[i_step][i_isotope],'Msun')

**Your task**. In the cell below, create arrays for plotting the following quantities:
* The total mass of gas in the circumgalactic medium (outside the galaxy)
* The total mass of gas inside galaxy
* The total mass of stars
    * **Warning!** You will need to account the mass ejected by stars.
    * m_star =  total_mass_of_stars_formed - total_mass_ejected_by_stars

In [ ]:
# Declaration of the arrays
m_gas_circumgalactic = np.zeros(o.inner.nb_timesteps) # [M_sun]
m_gas_galaxy = np.zeros(o.inner.nb_timesteps) # [M_sun]
m_star = np.zeros(o.inner.nb_timesteps) # [M_sun]

###
### Fill the arrays ###
###

#for i_step in range(o.inner.nb_timesteps):
    # ...

In [ ]:
###
### Fill this cell ###
###

# Write a script to test whether you filled the arrays correctly.
# You need to make sure the total mass is concerved.
# The total amount of matter should be the same (within 10% error) from one timestep to another.

In [ ]:
%matplotlib nbagg
plt.plot(t_sim, m_gas_circumgalactic)
plt.plot(t_sim, m_gas_galaxy)
plt.plot(t_sim, m_star)

###
### Add legend and axis labels
### 

### Your Exercise
* Re-run the model with star formation switched on.
    * Explore different "sfe" values between 0.001 and 0.05 (re-run OMEGA+, see above).
    * Understand the role of star formation, and try to predict what will happen when you change "sfe".
* Re-run the model with star formation and galactic outflows switched on.
    * Set "sfe = 0.01"
    * Explore different "mass_loading" values between 1.0 and 5.0.
    * Understand the role of galactic outflows, and try to predict what will happen when you change "mass_loading".
* Try to find a set of parameters that repect the following criteria:
    * About $6\times10^{10}$ M$_\odot$ of stars formed.
    * The mass of the circumgalactic medium must be larger than $6\times10^{10}$ M$_\odot$.

# 2. Calibration of the Milky Way Model

Here you will calibrate your simple Milky Way model against observations.  You only need to work with 2 parameters (star formation and galactic inflows). You need to vary the parameters until you respect all 3 observational constraints. You will then use this calibrated model to investigate the origin of the elements in our galaxy. 

** Useful information to help you calibrate your model **
* The star formation efficiency (sfe) mostly affects [Fe/H].
* The amount of galactic inflows (in_mag) mostly affect the total stellar mass formed.

### Run your model

In [ ]:
# Star formation efficiency.
# This is controling how fast the gas inside the galaxy is converted into stars.
# You can vary this parameter between 0.005 and 0.05
sfe = 0.005

# Magnitude (strength) of the galactic inflow rate.
# A higher "in_mag" value means more inflows.
# You can vary this parameter between 0.5 and 2.0
in_mag = 0.5

# !!Do not modify the following line!!
# Run OMEGA+ with the first set of parameters
o_mw = omega_plus.omega_plus(sfe=sfe, t_star=1.0e8, special_timesteps=90, \
                 exp_infall=[[in_mag*40, 0.0, 0.8e9], [in_mag*5, 1.0e9, 7.0e9]], \
                 table='yield_tables/agb_and_massive_stars_K10_K06_0.5HNe.txt', \
                 mass_loading=0.5, imf_yields_range=[1,40])

### Constraint #1 - Total stellar mass formed
Your model should form in total in between $4.5\times10^{10}$ and $5.0\times10^{10}$ M$_\odot$ of stars.

In [ ]:
# Print the total stellar mass formed
print("Total stellar mass formed:",'%.2e'%sum(o_mw.inner.history.m_locked),'M_sun')

### Constraint #2 - Final star formation rate
The final star formation rate (SFR) of your model should be within the cyan/blue vertical band (in between 0.6 and 3.0 M$_\odot$/yr).

In [ ]:
# Plot the evolution of the star formation rate (SFR)
%matplotlib nbagg
o_mw.inner.plot_star_formation_rate(color='r', marker='o', label='My model')
plt.xlabel('Galactic age [yr]')

# Plot the observational constraint (cyan color)
plt.plot([13e9,13e9], [0.65,3.0], linewidth=6, color='c', alpha=0.5)

### Constraint #3 - Solar value [Fe/H=0] by the time the Sun forms
Your prediction needs to go through the intersection between the horizontal and vertical dotted lines.

This is using the following spectroscopic notation,

$$\mathrm{[Fe/H]}=\log(n_\mathrm{Fe}/n_\mathrm{H})-\log(n_\mathrm{Fe}/n_\mathrm{H})_\odot$$

where n_X is the number density of the element X. [X/Y] ratios, where X and Y can be any elements, are expressed in log-space and are normalized to the solar composition.

In [ ]:
# Plot the evolution of [Fe/H], the iron abundance of the gas inside the galaxy.
%matplotlib nbagg
o_mw.inner.plot_spectro(color='r', marker='o', label='My model')
plt.xlabel('Galactic age [yr]')
plt.xscale('linear')
plt.ylim(-2,1)

# Plot the solar value (black dotted lines)
t_Sun = 13.0e9 - 4.6e9
plt.plot([t_Sun,t_Sun], [-2,1], ':k')
plt.plot([0,13e9], [0,0], ':k')

# 3. Contribution of Different Enrichment Sources

Now that you have your Milky Way model calibrated, you can start looking at the evolution of the elements (and also isotopes if you want).

### Select your element (or isotope)

In [ ]:
# Target element (or isotope)
# Element nomenclature: First letter in capital. e.g., C, N, O, Mg, ..
# Isotope nomenclature: O-16, Fe-56, ..
specie = 'N'

# Verify that the target specie is available
if specie in o_mw.inner.history.elements or specie in o.inner.history.isotopes:
    print('The element/isotope is available.')
else:
    print('Error - The element/isotope IS NOT available, please select a new one.')

### Plot the evolution of the selected element
This will show the mass of the element in form of gas **inside the galaxy**. It will also tell you which type of astrnomical events produce the most of this element (or isotope) in the Milky Way galaxy.

In [ ]:
%matplotlib nbagg
o_mw.inner.plot_mass(specie=specie, label='Total mass')
o_mw.inner.plot_mass(specie=specie, source='massive', label='Massive stars')
o_mw.inner.plot_mass(specie=specie, source='agb', label='Low-mass stars')
o_mw.inner.plot_mass(specie=specie, source='sn1a', label='Type Ia supernovae')
plt.xlabel('Galactic age [yr]')
plt.ylabel('Mass of '+specie)
#plt.xscale('linear')
#plt.ylim(1e3,6e7)

# 4. Chemical Evolution Trends [Mg/Fe] - vs - [Fe/H] 

Here you are comparing your Milky Way model against observations. This is for the evolution of elemental ratios as a function of time (or as a function of [Fe/H]..) throughout the history of the Milky Way.

In [ ]:
# Load the stellar abundance data
s = stellab.stellab()

# Create a list of reference observational papers.
obs = ['stellab_data/milky_way_data/Cohen_et_al_2013_stellab',\
       'stellab_data/milky_way_data/Jacobson_et_al_2015_stellab',\
       'stellab_data/milky_way_data/Venn_et_al_2004_stellab',\
       'stellab_data/milky_way_data/Bensby_et_al_2014_stellab',\
       'stellab_data/milky_way_data/Nissen_et_al_2014_stellab',\
       'stellab_data/milky_way_data/Battistini_Bensby_2015_stellab',\
       'stellab_data/milky_way_data/Battistini_Bensby_2016_stellab']

In [ ]:
# Select your abundance ratios.
xaxis = '[Fe/H]'
yaxis = '[Mg/Fe]'

# Plot data using your selection of data points
%matplotlib nbagg
s.plot_spectro(xaxis=xaxis, yaxis=yaxis, norm='Asplund_et_al_2009', obs=obs)

# Extract the numerical prediction of OMEGA+ using the "return_x_y" argument.
xy_o = o_mw.inner.plot_spectro(xaxis=xaxis, yaxis=yaxis, return_x_y=True, solar_norm='Asplund_et_al_2009')
plt.plot(xy_o[0], xy_o[1], color='w', linewidth=3.0)
plt.plot(xy_o[0], xy_o[1], color='k', linewidth=1.5, label='Your model')

# Set the range of the x and y axis
plt.legend(loc='center left', bbox_to_anchor=(1.01, 0.5), markerscale=0.8, fontsize=14)
plt.xlim(-4.5,0.75)
plt.ylim(-1.0,1.)

### Your Exercise
* 1) Understand why **[Mg/Fe]-vs-[Fe/H]** is bending down and decreasing at [Fe/H] > -1.
    * The analysis plots shown in the previous sections will be useful.
* 2) Understand why **[C/O]-vs-[Fe/H]** is increasing at [Fe/H] > -2.
    * Don't worry too much if the fit is not perfect.
* 3) Understand why **[Ca/Si]-vs-[Fe/H]** is increasing at [Fe/H] > -2.
    * Don't worry too much if the fit is not perfect.
* 4) Can you explain why **[Ti/Fe]-vs-[Fe/H]** does not fit?
    * By only looking at the observed data, can you guess in which stars Ti should be produced?

In [ ]:
# Make your plots and analysis here ..
# Also write you answers

# 5. The Origin of "Light" Elements in our Solar System

Besides stellar abundances data, the abundance distribution of the Sun can be used to understand the origin of the elements. The solar composition represents only one timeframe in the history of the Milky Way, but the advantage is that there is data for all the stable elements and isotopes. Here you will compare your chemical evolution predictions with the solar composition.

In [ ]:
# Load the functions for extracting abundances distributions
%run script_solar_ab.py

## 5.1 The Role of Type Ia Supernovae

Here you will vary the number of Type Ia Supernovae until you match the solar composition.

### Run the model

In [ ]:
# Number of Type Ia supernova per units of stellar mass formed.
# You can vary this parameters between 1.0e-4 and 1.0e-1
nb_1a_per_m = 1e-1

# Run OMEGA+ with your set of parameters (simple Milky Way model).
o_51 = omega_plus.omega_plus(sfe=sfe, t_star=1.0e8, special_timesteps=120, \
                 exp_infall=[[in_mag*40, 0.0, 0.8e9], [in_mag*5, 1.0e9, 7.0e9]], \
                 table='yield_tables/agb_and_massive_stars_K10_K06_0.5HNe.txt', \
                 mass_loading=0.5, imf_yields_range=[1,40], nb_1a_per_m=nb_1a_per_m)

In [ ]:
###
### Fill the cell ###
###

# Write a script to find the timestep where the Sun should form.
# You need to look in the metallicity array and find the timestep
# index where the metallicity is the closest to the solar value (0.014).
# Don't find it by hand, write a script to find it.
#
# Metallicity array --> o_51.inner.history.metallicity[ timestep index ]

# Timestep index where the Sun should aproximately form.
# i_t_Sun = place your index here

In [ ]:
# Get source contributions.
# This will use the i_t_Sun parameter found in the previous cell.
m_el_all, m_el_agb, m_el_massive, m_el_sn1a, m_el_nsm = \
    get_individual_sources(o_51.inner, i_step_sol=i_t_Sun)

### Plot the abundance distribution
You do not need to change anything in the cell below.

In [ ]:
# Define the figure size
%matplotlib nbagg
fig = plt.figure(figsize=(8,4.0))
matplotlib.rcParams.update({'font.size': 14.0})

# Plot solar abundance data
plt.plot(solar_Z, solar_ab, color='k', marker='o', linewidth=4, alpha=0.5, label='Solar')

# All sources combined
plt.plot(Z_charge, m_el_all, color='orange', label='All sources', alpha=1.0, linestyle='-', linewidth=2)

# Type Ia supernovae
plt.plot(Z_charge, m_el_sn1a, color='g', label='SNe Ia', alpha=0.8, linestyle='-', marker='s')

# Massive stars (core-collapse supernovae)
plt.plot(Z_charge, m_el_massive, color='b', label='Massive', alpha=0.8, linestyle='-', marker='^')

# Add element annotations
for i in range(0,nb_elements):
    plt.annotate(elements[i], xy=(solar_Z[i],yy[i]), color='k',\
                 fontsize=13, ha='center', va='center')

# Label and axis
plt.legend(fontsize=13, loc='center left', bbox_to_anchor=(1, 0.5))
plt.xlabel('Z (charge number)', fontsize=16)
plt.ylabel('X (mass fraction)', fontsize=16)
plt.xlim(19.5,30.5)
plt.ylim(1e-9,3e-1)
plt.yscale('log')

# Frame tuning
plt.subplots_adjust(top=0.95)
plt.subplots_adjust(right=0.75)
plt.subplots_adjust(left=0.15)
plt.subplots_adjust(bottom=0.14)

### Your Exercise
* 1) Modify the number of Type Ia supernovae in your model until your reproduce the solar composition for **Fe, Co, and Ni**.
* 2) How many Type Ia supernovae you need in total in the Milky Way to have enough Fe, Co, and Ni?
    * The number of Type Ia supernovae occuring during each timestep is stored in the following array.
    * **o_51.inner.history.sn1a_numbers**[ timestep index ]

# 6. The Origin of Neutron-Capture Elements in our Solar System

In the following, you will assume that neutron star mergers are the only source of r-process elements in the Milky Way.  This may not be true, as core-collapse supernovae from massive stars could also contribute.

In [ ]:
# Number of neutron star mergers per units of stellar mass formed.
# You can vary this parameters between 1.0e-6 and 1.0e-4
nb_nsm_per_m = 1.0e-6

# Run OMEGA+ with your set of parameters (simple Milky Way model).
r_process_yields = 'yield_tables/r_process_arnould_2007.txt'
o_6 = omega_plus.omega_plus(sfe=sfe, t_star=1.0e8, special_timesteps=120, \
                 exp_infall=[[in_mag*40, 0.0, 0.8e9], [in_mag*5, 1.0e9, 7.0e9]], \
                 mass_loading=0.5, imf_yields_range=[1,40], nb_1a_per_m=nb_1a_per_m, \
                 ns_merger_on=True, nb_nsm_per_m=nb_nsm_per_m, t_nsm_coal=1.0e8, \
                 nsmerger_table=r_process_yields)

# Get source contributions (this needs to be right after running OMEGA+)
m_el_all_6, m_el_agb_6, m_el_massive_6, m_el_sn1a_6, m_el_nsm_6 = \
    get_individual_sources(o_6.inner, i_step_sol=i_t_Sun)

In [ ]:
# Define the figure size
%matplotlib nbagg
fig = plt.figure(figsize=(8,4.0))
matplotlib.rcParams.update({'font.size': 14.0})

# Plot solar abundance data
plt.plot(solar_Z, solar_ab, color='k', marker='o', linewidth=4, alpha=0.5, label='Solar')

# All sources combined
plt.plot(Z_charge, m_el_all_6, color='orange', label='All sources', alpha=1.0, linestyle='-', linewidth=2)

# AGB stars
plt.plot(Z_charge, m_el_agb_6, color='r', label='AGB', alpha=0.8, linestyle='-', marker='x')

# Neutron star mergers (r-process)
plt.plot(Z_charge, m_el_nsm_6, color='c', label='NS mergers', alpha=0.8, linestyle='-', marker='^')

# Add element annotations
for i in range(0,nb_elements):
    plt.annotate(elements[i], xy=(solar_Z[i],yy[i]), color='k',\
                 fontsize=13, ha='center', va='center')

# Label and axis
plt.legend(fontsize=13, frameon=False)
plt.xlabel('Z (charge number)', fontsize=16)
plt.ylabel('X (mass fraction)', fontsize=16)
plt.xlim(30.5,82.5)
plt.ylim(1e-11,1e-4)
plt.yscale('log')

# Frame tuning
plt.subplots_adjust(top=0.95)
plt.subplots_adjust(right=0.95)
plt.subplots_adjust(left=0.15)
plt.subplots_adjust(bottom=0.14)

### Your Exercise
* 1) Modify the number of neutron star merger in your model until your reproduce the solar composition for **Te, I, Xe, Os, Ir, Pt, and Au**.  You can zoom-in in the plot above
* 2) How many neutron star merger you need in total in the Milky Way to have enough gold (Au)?
    * The number of neutron star merger occuring during each timestep is stored in the following array.
    * **o_6.inner.history.nsm_numbers**[ timestep index ]

# 7. Isotopic Composition of the Sun

At the time the Sun forms in your model, when you fit the abundance of an element, do you also fit the isotopic distribution for that element?

If you are interested in this task, and if you have time, raise your hand and I will explain.

In [ ]:
# Extract the list of isotopes mass fraction for the Sun
iso_A09_name = []
iso_A09_f = []
with open('Asplund_et_al_2009_iso.txt') as ff:
    for line in ff:
        line_split = [str(x) for x in line.split()]
        iso_A09_name.append(line_split[0])
        iso_A09_f.append(float(line_split[1])/100.0)
ff.close()

In [ ]:
# For each isotope in the Sun, this is the mass contribution to the total mass of the associated element.
# For example, "Ca-40 0.96941" means that the Ca-40 isotope represent 96.941% of the mass of the Ca element.
for i_iso in range(len(iso_A09_name)):
    print(iso_A09_name[i_iso], iso_A09_f[i_iso])